In [2]:
import pandas as pd
import numpy as np
import argparse
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.contrib.data.handler import Alpha158
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.dataset.handler import DataHandlerLP
from qlib.data.dataset.processor import Processor
from qlib.utils import get_callable_kwargs
from qlib.utils import flatten_dict
from dataclasses import dataclass
import os

In [3]:
directory_path = "/Users/yumengzhang/.qlib/qlib_data/cn_data"
if os.path.exists(directory_path):
    contents = os.listdir(directory_path)
    print(contents)
else:
    print("Directory does not exist.")

['calendars', 'features', 'instruments', '20231126172858_qlib_data_cn_1d_latest.zip']


In [4]:
args = {
    'start_time': '2010-12-01',
    'end_time': '2020-12-31',
    'fit_end_time': '2017-12-31',
    'val_start_time': '2018-01-01',
    'val_end_time': '2018-12-31',
    'seq_len': 20
}

In [5]:
provider_uri = "~/.qlib/qlib_data/cn_data" 
qlib.init(provider_uri=provider_uri, region=REG_CN)

[6640:MainThread](2023-11-26 18:32:27,404) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[6640:MainThread](2023-11-26 18:32:27,408) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[6640:MainThread](2023-11-26 18:32:27,409) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/yumengzhang/.qlib/qlib_data/cn_data')}


In [6]:
market = "csi300"
benchmark = "SH000300"

In [7]:
data_handler_config = {
    
    "start_time": f"{args['start_time']}",
    "end_time": f"{args['end_time']}",
    "fit_start_time": f"{args['start_time']}",
    "fit_end_time": f"{args['fit_end_time']}",
    "instruments": "csi300",
    "infer_processors": [{"class": 'RobustZScoreNorm', 'kwargs' : {'clip_outlier': True, 'fields_group': 'feature'}},
                            {'class': 'Fillna', 'kwargs': {'fields_group': 'feature'}}],
    "learn_processors": [{"class": 'DropnaLabel'},
                            {"class": 'CSRankNorm', 'kwargs' : {'fields_group': 'label'}}],
    }

In [8]:
dataset = Alpha158(**data_handler_config)

[6640:MainThread](2023-11-26 18:32:55,408) INFO - qlib.timer - [log.py:127] - Time cost: 22.428s | Loading data Done
/opt/anaconda3/envs/gen/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
[6640:MainThread](2023-11-26 18:33:11,035) INFO - qlib.timer - [log.py:127] - Time cost: 15.246s | RobustZScoreNorm Done
[6640:MainThread](2023-11-26 18:33:11,395) INFO - qlib.timer - [log.py:127] - Time cost: 0.359s | Fillna Done
[6640:MainThread](2023-11-26 18:33:11,651) INFO - qlib.timer - [log.py:127] - Time cost: 0.151s | DropnaLabel Done
/opt/anaconda3/envs/gen/lib/python3.11/site-packages/pyqlib-0.9.3.99-py3.11-macosx-10.9-x86_64.egg/qlib/data/dataset/processor.py:363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [9]:
dataframe = dataset.fetch(col_set=["feature","label"])

In [10]:
train = dataframe.loc[pd.IndexSlice[:args['fit_end_time'], :], :]

val_start_time = pd.to_datetime(args['val_start_time']) - pd.DateOffset(days=30)
valid = dataframe.loc[pd.IndexSlice[val_start_time: args['val_end_time'], :], :]

test_start_time = pd.to_datetime(args['val_end_time']) - pd.DateOffset(days=30)
test = dataframe.loc[pd.IndexSlice[test_start_time:, :], :]

In [11]:
train.to_pickle('data/train.pkl')
valid.to_pickle('data/valid.pkl')
test.to_pickle('data/test.pkl')

In [13]:
df_train = pd.read_pickle('data/train.pkl')
df_train

feature                                          \
                           KMID      KLEN     KMID2       KUP      KUP2   
datetime   instrument                                                     
2010-12-01 SH600000    0.383650 -0.755405  0.607150  0.008831  0.650947   
           SH600004    0.068085 -1.058037  0.151821 -0.460410  0.216933   
           SH600006   -0.453860 -0.934459 -0.867204 -0.994416 -1.085038   
           SH600008    3.000000  3.000000  0.961863  2.980869 -0.045844   
           SH600009    0.599052 -0.386866  0.700432  0.050170  0.250396   
...                         ...       ...       ...       ...       ...   
2017-12-29 SZ300124    0.378876 -0.402064  0.447818 -0.003429  0.195667   
           SZ300136    0.862173  0.050290  0.769642  0.530148  0.403011   
           SZ300144    0.065137 -0.948217  0.126475 -0.227788  0.542520   
           SZ300251   -0.115961 -0.411221 -0.137967  0.067124  0.295906   
           SZ300315    0.662231 -0.838009  1.138208 -0.994416 -1.085038   

                                                                         ...  \
                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...   
datetime   instrument                                                    ...   
2010-12-01 SH600000   -0.492484 -0.131389  0.165900  0.261354 -0.381240  ...   
           SH600004    0.172906  1.766418  0.235458  0.522648 -0.068008  ...   
           SH600006   -0.035812  0.953234 -0.098122 -0.186634  0.457281  ...   
           SH600008    1.390320 -0.478602  2.860721  0.680851 -3.000000  ...   
           SH600009   -0.161354  0.014838  0.431732  0.502503 -0.593196  ...   
...                         ...       ...       ...       ...       ...  ...   
2017-12-29 SZ300124    0.535045  0.864352  0.455085  0.535451 -0.376525  ...   
           SZ300136   -0.298109 -0.368340  0.462086  0.410622 -0.850095  ...   
           SZ300144    0.219071  1.489757  0.168991  0.326631 -0.065067  ...   
           SZ300251    1.233994  1.723403  0.200576  0.237556  0.116182  ...   
           SZ300315   -0.403395  0.106053  0.763613  1.306490 -0.655082  ...   

                                                                         \
                        VSUMN10   VSUMN20   VSUMN30   VSUMN60    VSUMD5   
datetime   instrument                                                     
2010-12-01 SH600000    1.365201  3.000000  2.909312 -0.129174 -0.539058   
           SH600004    0.908592  2.271337  1.387111 -0.001687 -1.085363   
           SH600006    0.575351 -0.026397  2.796952 -0.282580 -0.971235   
           SH600008   -0.828523 -1.109672 -0.990983 -1.547579  0.880282   
           SH600009    1.592773  1.035751  2.208660 -0.079021 -0.663230   
...                         ...       ...       ...       ...       ...   
2017-12-29 SZ300124   -0.865725  0.331437  1.183401  0.009151  0.024263   
           SZ300136    0.617357  2.052885  0.452057  0.027726 -1.397350   
           SZ300144    0.069872  0.294803  0.696110 -0.111176 -1.926775   
           SZ300251    0.584108  0.642116  0.897898  0.909328  0.224836   
           SZ300315    0.096395  1.993828  1.743830  0.015848  0.067828   

                                                                  label  
                        VSUMD10   VSUMD20   VSUMD30   VSUMD60    LABEL0  
datetime   instrument                                                    
2010-12-01 SH600000   -1.365201 -3.000000 -2.909313  0.129175  0.002346  
           SH600004   -0.908592 -2.271338 -1.387111  0.001689 -0.004453  
           SH600006   -0.575351  0.026398 -2.796953  0.282580  0.009363  
           SH600008    0.828522  1.109674  0.990983  1.547577       NaN  
           SH600009   -1.592774 -1.035752 -2.208659  0.079021  0.013029  
...                         ...       ...       ...       ...       ...  
2017-12-29 SZ300124    0.865724 -0.331436 -1.183401 -0.009151 -0.005074  
           SZ300136   -0.617358 -2.052886 -0.452056 -0.027725

In [14]:
df_valid = pd.read_pickle('data/valid.pkl')
df_valid

feature                                          \
                           KMID      KLEN     KMID2       KUP      KUP2   
datetime   instrument                                                     
2017-12-04 SH600000    0.235835 -1.285156  0.758803 -0.747670 -0.216981   
           SH600008   -1.472306  0.172062 -1.233054 -0.401956 -0.542512   
           SH600009   -0.157936 -0.627264 -0.222585  0.132203  0.651039   
           SH600010    0.461626 -0.292345  0.505864  0.213028  0.361691   
           SH600015    0.130688 -1.087556  0.303526 -0.139871  1.085019   
...                         ...       ...       ...       ...       ...   
2018-12-28 SZ300144    1.100033  0.562393  0.768923 -0.994416 -1.085038   
           SZ300251   -2.379051  2.457106 -0.922464  1.815823  0.106385   
           SZ300296    0.557652 -0.238999  0.590180  0.464176  0.602804   
           SZ300408   -0.321190 -0.595205 -0.440598 -0.807732 -0.805033   
           SZ300433   -2.417423  1.316865 -1.280478 -0.526042 -0.813776   

                                                                         ...  \
                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...   
datetime   instrument                                                    ...   
2017-12-04 SH600000   -0.646831  0.343249  0.244746  0.783883 -0.234922  ...   
           SH600008   -0.733605 -0.783468 -1.371217 -1.143166  1.508901  ...   
           SH600009    0.418119  1.070692 -0.086920 -0.121941  0.158349  ...   
           SH600010    0.336641  0.501420  0.399227  0.435498 -0.458142  ...   
           SH600015   -0.479305  0.343285  0.000012  0.000024 -0.130409  ...   
...                         ...       ...       ...       ...       ...  ...   
2018-12-28 SZ300144    2.208889  1.260468  1.877616  1.306490 -1.080450  ...   
           SZ300251    0.333048 -0.521837 -2.588394 -0.999065  2.476085  ...   
           SZ300296   -0.102586 -0.025716  0.275586  0.290338 -0.552574  ...   
           SZ300408    1.111155  1.980769  0.277771  0.379307  0.322899  ...   
           SZ300433   -0.256353 -0.635206 -2.013191 -1.061506  2.517673  ...   

                                                                         \
                        VSUMN10   VSUMN20   VSUMN30   VSUMN60    VSUMD5   
datetime   instrument                                                     
2017-12-04 SH600000    0.673248 -0.423079 -0.187926 -0.118766 -2.022600   
           SH600008   -0.182618  0.149068  1.212574  2.515507 -1.743458   
           SH600009    1.415942  1.617286  0.456565 -0.294937 -1.249788   
           SH600010   -1.358907 -0.454561 -1.578056  2.269395  0.720146   
           SH600015    0.815739  0.017464 -0.287332  0.400839 -1.234709   
...                         ...       ...       ...       ...       ...   
2018-12-28 SZ300144   -0.893180 -0.676162 -0.423588  0.280176  0.760944   
           SZ300251   -1.435888 -1.227484  0.085326 -0.351187  0.594816   
           SZ300296    1.801298  1.320697  0.890979  0.815438 -0.014210   
           SZ300408    0.254329  0.403555  0.868906  0.607359 -0.167192   
           SZ300433   -1.697815 -1.568577  0.029410 -1.642565  1.436140   

                                                                  label  
                        VSUMD10   VSUMD20   VSUMD30   VSUMD60    LABEL0  
datetime   instrument                                                    
2017-12-04 SH600000   -0.673248  0.423080  0.187926  0.118766 -0.015945  
           SH600008    0.182617 -0.149068 -1.212575 -2.515500  0.005825  
           SH600009   -1.415943 -1.617287 -0.456564  0.294938 -0.033840  
           SH600010    1.358906  0.454562  1.578057 -2.269389  0.000000  
           SH600015   -0.815740 -0.017463  0.287332 -0.400838 -0.010537  
...                         ...       ...       ...       ...       ...  
2018-12-28 SZ300144    0.893180  0.676163  0.423588 -0.280175 -0.026639  
           SZ300251    1.435888  1.227485 -0.085326  0.351187

In [15]:
df_test = pd.read_pickle('data/test.pkl')
df_test

feature                                          \
                           KMID      KLEN     KMID2       KUP      KUP2   
datetime   instrument                                                     
2018-12-03 SH600000    0.165710 -0.536890  0.216804 -0.560995 -0.465018   
           SH600008    0.171480 -0.492957  0.216800 -0.545895 -0.465016   
           SH600009   -0.282892  0.226761 -0.230521  1.317883  0.975168   
           SH600010    0.783283 -0.094838  0.758803  0.029959  0.000004   
           SH600011    2.114516  1.554480  1.040646  1.540481  0.279008   
...                         ...       ...       ...       ...       ...   
2020-09-25 SZ300413    0.904611  0.075797  0.796533  0.674311  0.521550   
           SZ300433   -0.958482  0.286111 -0.758798 -0.846933 -0.957375   
           SZ300498   -0.743196 -0.505190 -0.948506 -0.216862 -0.000002   
           SZ300601   -1.287670  2.360488 -0.510888  3.000000  0.896579   
           SZ300628   -0.316698 -0.209702 -0.328988 -0.433256 -0.447656   

                                                                         ...  \
                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...   
datetime   instrument                                                    ...   
2018-12-03 SH600000    1.463788  2.308705  0.716539  0.933212 -0.165260  ...   
           SH600008    1.552470  2.308716  0.741501  0.933211 -0.170994  ...   
           SH600009    1.054937  0.691583 -0.407745 -0.330745  0.284213  ...   
           SH600010    0.121623  0.106066  0.677401  0.653249 -0.773301  ...   
           SH600011   -1.081524 -1.079974  0.990540  0.485277 -2.043333  ...   
...                         ...       ...       ...       ...       ...  ...   
2020-09-25 SZ300413   -0.525728 -0.581976  0.387044  0.339256 -0.891325  ...   
           SZ300433    1.689805  1.152573 -0.097513 -0.076843  0.973857  ...   
           SZ300498   -0.624898 -0.486957 -0.771268 -0.979855  0.752406  ...   
           SZ300601    1.348346 -0.098962 -1.939902 -0.766157  1.315571  ...   
           SZ300628    1.774474  1.939030  0.344570  0.356318  0.318355  ...   

                                                                         \
                        VSUMN10   VSUMN20   VSUMN30   VSUMN60    VSUMD5   
datetime   instrument                                                     
2018-12-03 SH600000   -3.000000 -3.000000  0.190599 -0.383691  2.892849   
           SH600008    2.156863 -0.105260 -0.037513 -0.766494 -0.891044   
           SH600009   -1.438813  0.850300  0.258306 -0.883333  2.887834   
           SH600010   -0.622036 -0.278566  0.131200 -0.669504  1.522675   
           SH600011   -0.117814 -0.853466  0.871683 -0.558842  0.279279   
...                         ...       ...       ...       ...       ...   
2020-09-25 SZ300413    1.902129  2.807487  0.368912  0.777892  0.462059   
           SZ300433    3.000000  1.516506  0.275959  1.510199 -1.363506   
           SZ300498    0.497830  3.000000  0.138158  0.938714 -0.806903   
           SZ300601    0.038803  0.225010  0.585177 -0.062209  0.249203   
           SZ300628    0.528503  0.250647  0.532218  0.657884 -2.781974   

                                                                  label  
                        VSUMD10   VSUMD20   VSUMD30   VSUMD60    LABEL0  
datetime   instrument                                                    
2018-12-03 SH600000    3.000000  3.000000 -0.190600  0.383691 -0.007220  
           SH600008   -2.156864  0.105261  0.037514  0.766494  0.013699  
           SH600009    1.438813 -0.850301 -0.258307  0.883332 -0.008257  
           SH600010    0.622036  0.278567 -0.131200  0.669503 -0.012658  
           SH600011    0.117813  0.853468 -0.871684  0.558842  0.001377  
...                         ...       ...       ...       ...       ...  
2020-09-25 SZ300413   -1.902129 -2.807489 -0.368913 -0.777889       NaN  
           SZ300433   -3.000000 -1.516507 -0.275958 -1.510195